In [1]:
from koselleck import *

In [2]:
df=read_df(FN_VECTOR_SCORES_RUNS)
df

word  Abs-Conc.Median  Woman-Man.VG  \
corpus period    run                                                    
bpo    1680-1690 run_01           lane        -2.483134     -0.235502   
                 run_01         sermon        -2.388534     -0.701400   
                 run_01         street        -2.292043     -0.118967   
                 run_01            sir        -2.276923     -2.038116   
                 run_01           gate        -2.240158     -0.138460   
...                                ...              ...           ...   
       1960-1970 run_25      authority         2.813621     -1.264022   
                 run_25  understanding         2.885293     -0.001055   
                 run_25       attitude         2.911559     -0.122687   
                 run_25       politics         2.912478     -0.531165   
                 run_25      criticism         3.016368     -1.049019   

                         Human-Object.VG  Pleasure-Pain.HGI  Interp-Desc.HGI  \
corpus period    run                                                           
bpo    1680-1690 run_01        -0.186116          -1.866360        -0.265112   
                 run_01         0.024315          -2.054366        -0.271518   
                 run_01        -0.473321          -1.796022        -0.301607   
                 run_01         2.185661          -2.329914        -1.539692   
                 run_01        -0.640849          -1.494840        -0.690724   
...                                  ...                ...              ...   
       1960-1970 run_25         1.597309           0.147791         2.549739   
                 run_25         1.568625           2.504389         2.551782   
                 run_25         1.671437           0.448850         1.943168   
                 run_25         2.264574           0.624442         2.120086   
                 run_25         1.541799           0.169381         2.460823   

                         Polit-Acad.HGI  Qual-Quant.HGI  Time-Space.HGI  \
corpus period    run                                                      
bpo    1680-1690 run_01       -0.917638        1.616664       -1.422240   
                 run_01       -0.858428        1.649853       -1.495062   
                 run_01       -0.611056        1.118064       -1.210363   
                 run_01       -2.976701        2.321312        0.334780   
                 run_01       -0.853216        1.013274       -1.340083   
...                                 ...             ...             ...   
       1960-1970 run_25        0.962190       -1.272990        0.617229   
                 run_25       -1.250048       -1.274672        0.504747   
                 run_25        0.885571       -1.464163        1.005001   
                 run_25        0.507098       -1.274545        1.267613   
                 run_25       -0.915959       -1.436501        1.137464   

                         Pos-Neg.HGI  Virtue-Vice.HGI  Collective-Indiv.RH  \
corpus period    run                                                         
bpo    1680-1690 run_01    -1.125153        -1.809059            -0.556282   
                 run_01    -1.185452        -1.719505            -0.443085   
                 run_01    -1.109309        -1.720516            -0.346353   
                 run_01    -2.602805        -3.090032             1.424151   
                 run_01    -1.283006        -1.482110            -0.431185   
...                              ...              ...                  ...   
       1960-1970 run_25     1.595054         1.083185             1.410013   
                 run_25     3.216744         2.592878             0.274030   
                 run_25     1.293012         0.753041             1.467733   
                 run_25     1.653909         0.599896             2.741370   
                 run_25     1.466207         0.779930             1.293149   

                         Judg-Perc.HGI  Strong-Weak.

In [3]:
for i,gdf in df.groupby('word'): pass

In [4]:
def compare_means(df,num_proc=1):
    odf=pmap_groups(
        do_compare_means,
        df.groupby('word'),
        num_proc=num_proc
    ).reset_index()
    return odf.set_index(['vector','word']).sort_values('mean_diff_t_abs',ascending=False)
    
def do_compare_means(gdf,before=1770, after=1830, ymin=1700, ymax=1900, truns=25, trunsize=100):
    gdf=gdf.reset_index()
    gdf['year']=[int(str(period)[:4]) for period in gdf.period]
    gdf=gdf.query(f'{ymin}<=year<{ymax}')
    
    gdf_before = gdf.query(f'year<{before}')
    gdf_after = gdf.query(f'year>={after}')
    if not len(gdf_before) or not len(gdf_after): return pd.DataFrame()
    
    qcols = [c for c in gdf.select_dtypes('number').columns if c[0]==c[0].upper()]
    
    o=[]
    for qcol in qcols:
        for trun in range(truns):
            try:
                vals1=gdf_before[qcol].dropna()
                vals2=gdf_after[qcol].dropna()
                if not len(vals1) or not len(vals2): continue
                vals1=vals1.sample(n=trunsize,replace=True)
                vals2=vals2.sample(n=trunsize,replace=True)
            except ValueError as e:
                print(gdf.year.describe())
                display(gdf_before)
                display(gdf_after)
                
                print('!!',e)
                stop
                continue
            t,p=ttest_ind(vals1.values, vals2.values)
            o+=[{
                'vector':qcol,
                'mean1':vals1.mean(),
                'mean2':vals2.mean(),
                'mean_diff':vals2.mean() - vals1.mean(),
                'mean_diff_t':t,
                'mean_diff_p':p,
                'mean_diff_t_abs':abs(t),
                'len1':len(vals1),
                'len2':len(vals2),
                'trun':trun,
                'year1':gdf_before.year.median(),
                'year2':gdf_after.year.median(),
            }]
    if not o: return pd.DataFrame()
    odf=pd.DataFrame(o).groupby('vector').mean().reset_index()
    return odf.sort_values('mean_diff_t_abs',ascending=False).dropna()

In [5]:
odf = compare_means(df, num_proc=7)

Mapping do_compare_means [x7]: 100%|██████████| 6010/6010 [08:00<00:00, 12.50it/s]


In [6]:

odf.to_csv(FN_VECTOR_SCORES_DIFFMEANS)

In [7]:
odf

mean1     mean2  mean_diff   mean_diff_t  \
vector             word                                                     
Sing-Plural.M      patronage -0.204667  0.334828   0.539496 -4.834986e+16   
                   guile     -0.334521 -0.340682  -0.006161  3.491972e+14   
                   craftsman  0.187390 -0.369967  -0.557357  6.825422e+02   
                   haft      -0.062734 -0.362382  -0.299648  5.731582e+02   
                   task      -0.162905  0.928315   1.091221 -3.289679e+02   
...                                ...       ...        ...           ...   
Ambig.NX           earldom    0.062711  0.078337   0.015626 -2.210541e-01   
Active-Passive.HGI worsted    0.846115  0.849801   0.003686 -6.255704e-02   
Qual-Quant.HGI     nap        0.642011  0.639181  -0.002830  3.991275e-02   
Polit-Acad.HGI     diadem     1.744927  1.740579  -0.004348  2.712320e-02   
Time-Space.HGI     target    -1.053906 -1.047475   0.006430 -6.896979e-02   

                                mean_diff_p  mean_diff_t_abs   len1   len2  \
vector             word                                                      
Sing-Plural.M      patronage   0.000000e+00     4.834986e+16  100.0  100.0   
                   guile       0.000000e+00     3.491972e+14  100.0  100.0   
                   craftsman   0.000000e+00     6.825422e+02  100.0  100.0   
                   haft       6.788169e-316     5.731582e+02  100.0  100.0   
                   task       1.569778e-263     3.289679e+02  100.0  100.0   
...                                     ...              ...    ...    ...   
Ambig.NX           earldom     6.629309e-01     4.877848e-01  100.0  100.0   
Active-Passive.HGI worsted     6.495256e-01     4.840086e-01  100.0  100.0   
Qual-Quant.HGI     nap         6.499182e-01     4.779457e-01  100.0  100.0   
Polit-Acad.HGI     diadem      6.853909e-01     4.580874e-01  100.0  100.0   
Time-Space.HGI     target      6.890967e-01     4.474830e-01  100.0  100.0   

                              trun   year1   year2  
vector             word                             
Sing-Plural.M      patronage  12.0  1740.0  1860.0  
                   guile      12.0  1745.0  1860.0  
                   craftsman  12.0  1745.0  1860.0  
                   haft       12.0  1730.0  1860.0  
                   task       12.0  1730.0  1860.0  
...                            ...     ...     ...  
Ambig.NX           earldom    12.0  1755.0  1860.0  
Active-Passive.HGI worsted    12.0  1745.0  1860.0  
Qual-Quant.HGI     nap        12.0  1740.0  1860.0  
Polit-Acad.HGI     diadem     12.0  1740.0  1860.0  
Time-Space.HGI     target     12.0  1745.0  1860.0  

[87765 rows x 11 columns]

In [8]:
odf.head(25)

mean1     mean2  mean_diff   mean_diff_t  \
vector        word                                                        
Sing-Plural.M patronage    -0.204667  0.334828   0.539496 -4.834986e+16   
              guile        -0.334521 -0.340682  -0.006161  3.491972e+14   
              craftsman     0.187390 -0.369967  -0.557357  6.825422e+02   
              haft         -0.062734 -0.362382  -0.299648  5.731582e+02   
              task         -0.162905  0.928315   1.091221 -3.289679e+02   
Freq.M        case         -0.324262  5.334777   5.659039 -2.784541e+02   
Sing-Plural.M rental        0.001246 -0.317885  -0.319131  2.743391e+02   
              tact         -0.336786 -0.088881   0.247904 -2.673402e+02   
              yarn         -0.065486 -0.323998  -0.258512  2.535526e+02   
              rest         -0.083292  3.121321   3.204614 -2.518319e+02   
              mercury       0.008391 -0.207053  -0.215444  2.500106e+02   
Freq.M        rest         -0.107990  3.005771   3.113761 -2.360126e+02   
Sing-Plural.M ceiling      -0.335276 -0.203878   0.131398 -2.051453e+02   
Freq.M        childe       -0.212408 -0.367062  -0.154654  2.021619e+02   
              coincidence  -0.359743 -0.281169   0.078573 -1.900603e+02   
Sing-Plural.M fib          -0.246526 -0.351497  -0.104970  1.878076e+02   
              centre       -0.203912  0.759705   0.963617 -1.847676e+02   
              cert         -0.188016 -0.340052  -0.152036  1.837578e+02   
              caw          -0.246553 -0.345266  -0.098713  1.821737e+02   
              rad          -0.222346 -0.319311  -0.096965  1.794549e+02   
Freq.M        arrangement  -0.277178  0.515200   0.792378 -1.769054e+02   
Sing-Plural.M case         -0.351983  3.645976   3.997959 -1.751507e+02   
Freq.M        discussion   -0.329111  0.362807   0.691918 -1.695412e+02   
              task         -0.195351  0.814740   1.010092 -1.677373e+02   
Sing-Plural.M renunciation -0.206754 -0.328670  -0.121916  1.630177e+02   

                              mean_diff_p  mean_diff_t_abs   len1   len2  \
vector        word                                                         
Sing-Plural.M patronage      0.000000e+00     4.834986e+16  100.0  100.0   
              guile          0.000000e+00     3.491972e+14  100.0  100.0   
              craftsman      0.000000e+00     6.825422e+02  100.0  100.0   
              haft          6.788169e-316     5.731582e+02  100.0  100.0   
              task          1.569778e-263     3.289679e+02  100.0  100.0   
Freq.M        case          1.101080e-245     2.784541e+02  100.0  100.0   
Sing-Plural.M rental        7.849936e-248     2.743391e+02  100.0  100.0   
              tact          7.025033e-246     2.673402e+02  100.0  100.0   
              yarn          5.025363e-241     2.535526e+02  100.0  100.0   
              rest          1.774628e-245     2.518319e+02  100.0  100.0   
              mercury       2.554433e-246     2.500106e+02  100.0  100.0   
Freq.M        rest          2.579095e-241     2.360126e+02  100.0  100.0   
Sing-Plural.M ceiling       5.788152e-227     2.051453e+02  100.0  100.0   
Freq.M        childe        2.497666e-213     2.021619e+02  100.0  100.0   
              coincidence   6.346453e-219     1.900603e+02  100.0  100.0   
Sing-Plural.M fib           2.656182e-223     1.878076e+02  100.0  100.0   
              centre        1.130533e-219     1.847676e+02  100.0  100.0   
              cert          2.425649e-216     1.837578e+02  100.0  100.0   
              caw           9.485785e-217     1.821737e+02  100.0  100.0   
              rad           4.280429e-211     1.794549e+02  100.0  100.0   
Freq.M        arrangement   2.141628e-216     1.769054e+02  100.0  100.0   
Sing-Plural.M case          1.182056e-208     1.751507e+02  100.0  100.0   
Freq.M        discussion    5.858591e-210     1.695412e+02  100.0  100.0   
              task          1.076090e-209     1.677373e+02  100.0  100.0   
Sing-Plural.M renunciation  4

In [9]:
odf.groupby('vector').head(5).sort_values(['vector','mean_diff_t_abs'], ascending=[1,0])

mean1     mean2  mean_diff  mean_diff_t  \
vector          word                                                     
Abs-Conc.Median bracelet    0.123219 -1.622303  -1.745522    88.831909   
                anomaly    -0.077866  1.754484   1.832350   -88.359073   
                reaction   -0.386081  1.701870   2.087951   -85.817279   
                tact       -0.567576  1.556400   2.123975   -80.669778   
                precision   1.771678  0.874067  -0.897610    78.013188   
...                              ...       ...        ...          ...   
Woman-Man.VG    sportsman   0.693558 -4.384029  -5.077588   153.955488   
                butcher     0.535929 -2.754778  -3.290707    94.596921   
                canning     1.773838 -2.276663  -4.050500    94.171689   
                spear       0.446100 -2.702556  -3.148656    91.224657   
                marguerite -0.698566  3.858161   4.556727   -82.101164   

                              mean_diff_p  mean_diff_t_abs   len1   len2  \
vector          word                                                       
Abs-Conc.Median bracelet    2.436384e-155        88.831909  100.0  100.0   
                anomaly     9.142246e-154        88.359073  100.0  100.0   
                reaction    3.585127e-153        85.817279  100.0  100.0   
                tact        5.907858e-147        80.669778  100.0  100.0   
                precision   2.092149e-145        78.013188  100.0  100.0   
...                                   ...              ...    ...    ...   
Woman-Man.VG    sportsman   9.617438e-201       153.955488  100.0  100.0   
                butcher     6.268044e-161        94.596921  100.0  100.0   
                canning     9.050025e-158        94.171689  100.0  100.0   
                spear       2.099836e-159        91.224657  100.0  100.0   
                marguerite  2.719228e-149        82.101164  100.0  100.0   

                            trun   year1   year2  
vector          word                              
Abs-Conc.Median bracelet    12.0  1750.0  1860.0  
                anomaly     12.0  1740.0  1860.0  
                reaction    12.0  1750.0  1860.0  
                tact        12.0  1745.0  1860.0  
                precision   12.0  1755.0  1860.0  
...                          ...     ...     ...  
Woman-Man.VG    sportsman   12.0  1755.0  1860.0  
                butcher     12.0  1735.0  1860.0  
                canning     12.0  1755.0  1860.0  
                spear       12.0  1745.0  1860.0  
                marguerite  12.0  1740.0  1860.0  

[85 rows x 11 columns]

In [10]:
odf.loc['Abs-Conc.Median'].head(25)

,mean1,mean2,mean_diff,mean_diff_t,mean_diff_p,mean_diff_t_abs,len1,len2,trun,year1,year2
word,,,,,,,,,,,
bracelet,0.123219,-1.622303,-1.745522,88.831909,2.436384e-155,88.831909,100.0,100.0,12.0,1750.0,1860.0
anomaly,-0.077866,1.754484,1.832350,-88.359073,9.142246e-154,88.359073,100.0,100.0,12.0,1740.0,1860.0
reaction,-0.386081,1.701870,2.087951,-85.817279,3.585127e-153,85.817279,100.0,100.0,12.0,1750.0,1860.0
tact,-0.567576,1.556400,2.123975,-80.669778,5.907858e-147,80.669778,100.0,100.0,12.0,1745.0,1860.0
precision,1.771678,0.874067,-0.897610,78.013188,2.092149e-145,78.013188,100.0,100.0,12.0,1755.0,1860.0
apathy,0.499609,2.406515,1.906906,-74.652706,2.734092e-141,74.652706,100.0,100.0,12.0,1755.0,1860.0
fostering,0.176185,1.603007,1.426822,-74.122505,3.332758e-143,74.122505,100.0,100.0,12.0,1760.0,1860.0
admixture,-0.673244,0.607612,1.280856,-72.360645,6.811665e-139,72.360645,100.0,100.0,12.0,1750.0,1860.0
transportation,-0.716766,1.134434,1.851201,-69.911304,6.500884e-135,69.911304,100.0,100.0,12.0,1735.0,1860.0
